In [1]:
import pyodbc as odbc
import pandas as pd
from USER import DRIVERS,SERVER,DATABASE
import numpy as np

KHỞI TẠO KẾT NỐI VỚI SQL SERVER

In [2]:
conn_str = f"""
    DRIVER={DRIVERS};
    SERVER={SERVER};
    DATABASE={DATABASE};
    Trusted_Connection=yes;
    UID=sa;
    PWD=123456;
"""

conn = odbc.connect(conn_str)

In [3]:
cursor = conn.cursor()

NHẬP DỮ LIỆU VỀ ĐỊA PHƯƠNG VÀO SQL SERVER

In [4]:
data = pd.read_excel('Datasets/Clean_data/Merge_locality_data.xlsx')

In [5]:
data.columns

Index(['Vùng', 'Năm', 'Tỷ lệ nam nữ', 'Tổng số', 'Nam', 'Nữ',
       'Tổng dân số thành thị', 'Tổng dân số nông thôn', 'Diện tích(Km2)',
       'Dân số trung bình (Nghìn người)', 'Mật độ dân số (Người/km2)',
       'Tỷ suất sinh thô', 'Tỷ suất chết thô', 'Tỷ lệ tăng tự nhiên',
       'Tỷ suất nhập cư', 'Tỷ suất xuất cư', 'Tỷ suất di cư thuần'],
      dtype='object')

TẠO BẢNG LOCALITY_DATA TRÊN SQL

In [6]:
cursor = conn.cursor()
cursor.execute("""
IF OBJECT_ID('dbo.Locality_data','U') IS NULL
BEGIN
    CREATE TABLE Locality_data (
        region NVARCHAR(100) NULL,
        year INT NULL,
        sex_ratio FLOAT NULL,
        total_population FLOAT NULL,
        male_population FLOAT NULL,
        female_population FLOAT NULL,
        urban_population FLOAT NULL,
        rural_population FLOAT NULL,
        area_km2 FLOAT NULL,
        avg_population_thousand FLOAT NULL,
        population_density FLOAT NULL,
        crude_birth_rate FLOAT NULL,
        crude_death_rate FLOAT NULL,
        natural_increase_rate FLOAT NULL,
        immigration_rate FLOAT NULL,
        emigration_rate FLOAT NULL,
        net_migration_rate FLOAT NULL
    );
END
""")

conn.commit()
cursor.close()


THỰC HIỆN ÉP KIỂU ĐỂ CÓ THỂ ĐƯA LÊN SQL SERVER

In [7]:
data = data.replace('..',np.nan)

col = ['Tỷ lệ nam nữ', 'Tổng số', 'Nam', 'Nữ', 'Tổng dân số thành thị',
       'Tổng dân số nông thôn', 'Diện tích(Km2)', 'Dân số trung bình (Nghìn người)',
       'Mật độ dân số (Người/km2)', 'Tỷ suất sinh thô', 'Tỷ suất chết thô',
       'Tỷ lệ tăng tự nhiên', 'Tỷ suất nhập cư', 'Tỷ suất xuất cư',
       'Tỷ suất di cư thuần']

data[col] = data[col].apply(pd.to_numeric,errors='coerce')

data = data.replace(np.nan,None)
data['Vùng'] = data['Vùng'].astype('string')

C:\Users\Admin\AppData\Local\Temp\ipykernel_17984\2890863432.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data = data.replace('..',np.nan)


ĐƯA DỮ LIỆU VỀ ĐỊA PHƯƠNG LÊN SQL SERVER

In [8]:
cursor = conn.cursor()
for index,row in data.iterrows():
    cursor.execute("""
    INSERT INTO Locality_data (
        region, year, sex_ratio, total_population, male_population, 
        female_population, urban_population, rural_population, 
        area_km2, avg_population_thousand, population_density, 
        crude_birth_rate, crude_death_rate, natural_increase_rate, 
        immigration_rate, emigration_rate, net_migration_rate
    )
    VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)
    """,
        row['Vùng'],
        row['Năm'],
        row['Tỷ lệ nam nữ'],
        row['Tổng số'],
        row['Nam'],
        row['Nữ'],
        row['Tổng dân số thành thị'],
        row['Tổng dân số nông thôn'],
        row['Diện tích(Km2)'],
        row['Dân số trung bình (Nghìn người)'],
        row['Mật độ dân số (Người/km2)'],
        row['Tỷ suất sinh thô'],
        row['Tỷ suất chết thô'],
        row['Tỷ lệ tăng tự nhiên'],
        row['Tỷ suất nhập cư'],
        row['Tỷ suất xuất cư'],
        row['Tỷ suất di cư thuần']
    )

conn.commit()
cursor.close()

NHẬP DỮ LIỆU VỀ THÀNH THỊ NÔNG THÔN VÀO SQL SERVER

In [9]:
data2 = pd.read_excel('Datasets/Clean_data/Merge_urban_rural_data.xlsx')

In [10]:
data2.columns

Index(['Vùng', 'Năm', 'Tỷ số giới tính của dân số (Số nam/100 nữ)',
       'Tỷ số giới tính khi sinh (Số bé trai/100 bé gái)', 'Life_Exp',
       'Tỷ suất sinh thô', 'Tỷ suất chết thô', 'Tỷ lệ tăng tự nhiên'],
      dtype='object')

TẠO BẢNG URBAN_RURAL_DATA TRÊN SQL SERVER

In [11]:
cursor = conn.cursor()
cursor.execute("""
    IF OBJECT_ID('dbo.Urban_rural_data','U') IS NULL
    BEGIN
        CREATE TABLE Urban_rural_data (
            region NVARCHAR(100) NULL,
            year INT NULL,
            sex_ratio_population FLOAT NULL,
            sex_ratio_birth FLOAT NULL,
            life_expectancy FLOAT NULL,
            crude_birth_rate FLOAT NULL,
            crude_death_rate FLOAT NULL,
            natural_increase_rate FLOAT NULL
        );
    END
""")

conn.commit()
cursor.close()

ÉP KIỂU DỮ LIỆU ĐỂ CÓ THỂ ĐƯA LÊN SQL SERVER

In [12]:
data2['Vùng'] = data2['Vùng'].astype('string')
data2 = data2.replace(['..', '', ' '], np.nan)

num_cols = [
    'Tỷ số giới tính của dân số (Số nam/100 nữ)',
    'Tỷ số giới tính khi sinh (Số bé trai/100 bé gái)',
    'Life_Exp',
    'Tỷ suất sinh thô',
    'Tỷ suất chết thô',
    'Tỷ lệ tăng tự nhiên'
]

data2[num_cols] = data2[num_cols].apply(
    lambda col: pd.to_numeric(col.astype(str).str.replace(',', '.'), errors='coerce')
)

data2 = data2.replace(np.nan, None)
data2['Vùng'] = data2['Vùng'].astype('string')

C:\Users\Admin\AppData\Local\Temp\ipykernel_17984\3259959962.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data2 = data2.replace(['..', '', ' '], np.nan)


ĐƯA DỮ LIỆU VỀ THÀNH THỊ NÔNG THÔN LÊN SQL SERVER

In [13]:
cursor = conn.cursor()

for index,row in data2.iterrows():
    cursor.execute("""
    INSERT INTO Urban_rural_data (
            region, year, sex_ratio_population, sex_ratio_birth,
            life_expectancy, crude_birth_rate, crude_death_rate, natural_increase_rate
        )
        VALUES (?,?,?,?,?,?,?,?)
    """,
        row['Vùng'],
        row['Năm'],
        row['Tỷ số giới tính của dân số (Số nam/100 nữ)'],
        row['Tỷ số giới tính khi sinh (Số bé trai/100 bé gái)'],
        row['Life_Exp'],
        row['Tỷ suất sinh thô'],
        row['Tỷ suất chết thô'],
        row['Tỷ lệ tăng tự nhiên']  
    )

conn.commit()
cursor.close()

NHẬP DỮ LIỆU VỀ THẤT NGHIỆP VÀO SQL SERVER

In [14]:
data3 = pd.read_excel('Datasets/Clean_data/Unemployed_data.xlsx')

TẠO BẢNG UNEMPLOYED_DATA TRÊN SQL SERVER

In [15]:
cursor = conn.cursor()
cursor.execute('''
    IF OBJECT_ID('dbo.Unemployed','U') IS NULL
    BEGIN
        CREATE TABLE Unemployed_Data (
            Region nvarchar(100),	
            Year INT,
            Area nvarchar(100),
            Total FLOAT,
            Urban FLOAT,
            Rural FLOAT,
        )
    END
''')

conn.commit()

ÉP KIỂU DỮ LIỆU ĐỂ CÓ THỂ ĐƯA LÊN SQL SERVER

In [16]:
cols = ['Tổng số','Thành thị','Nông thôn']

data3[cols] = data3[cols].apply(pd.to_numeric,errors='coerce')
data3['Vùng'] = data3['Vùng'].astype('string')
data3['Khu vực'] = data3['Khu vực'].astype('string')

ĐƯA DỮ LIỆU VỀ THẤT NGHIỆP LÊN SQL SERVER

In [17]:
cursor = conn.cursor()
for index,row in data3.iterrows():
    cursor.execute("""
        INSERT INTO Unemployed_data (
            Region,Year,Area,Total,Urban,Rural) 
        VALUES(?,?,?,?,?,?)
        """,
        row['Vùng'],
        row['Năm'],
        row['Khu vực'],
        row['Tổng số'],
        row['Thành thị'],
        row['Nông thôn']
    )

conn.commit()